# Traitlets Tutorial
This tutorial shows how the Python package `traitlets` is used by PODPAC developers to make nodes. The same style should be used by new node developers.

In [1]:
import podpac
import traitlets as tl  # We use the shortform 'tl'


# A short, full-featured example
## Defining a PODPAC Node using Traitlets

In [12]:
# Create a new custom PODPAC Node
class MyNode(podpac.Node):
    # Define an input that's another specific type of Node
    input = tl.Instance(podpac.algorithm.CoordData, default_value=podpac.algorithm.CoordData())
    
    # Define attributes that affect the output
    dimension = tl.Unicode(default_value='time').tag(attr=True)  # The default for this trait is specified
    scale = tl.Float().tag(attr=True)
    
    # Define internal class attributes
    _other_attr = tl.Float(default_value=-1)  # The default is -1 for this trait
        
    # Define a default for this trait based on an equation
    # Note: This function is only called when the 'scale' attribute is first requested, and only if it hasn't been specified
    @tl.default('scale')
    def _default_scale(self):
        return self._other_attr ** 2
    
    # Set up an observer for the 'dimension' attribute using traits
    # Note: This function is called whenever the 'dimension' attribute is changed
    @tl.observe('dimension')
    def _modify_coord_data_dim(self, change):
        print ("Changing extracted dimension from %s to %s" % (change['old'], change['new']))
        self.input.coord_name = change['new']  # Change coord_name attribute of input node
        
    # Define the execute method, which is not implemented as part of the base PODPAC node
    def execute(self, coordinates, output=None, method=None):
        o = self.input.execute(coordinates)
        return o*scale
    
    # Define a function to print all of the class traits -- this is for nice display for this tutorial and not necessary for the node the function
    def __repr__(self):
        import json
        return json.dumps(self.base_definition(), indent=2)
# input = podpac.algorithm.CoordData()
my_node = MyNode()
print ("Note that tagged attributes are automatically included in the JSON representation of the Node.")
my_node

Note that tagged attributes are automatically included in the JSON representation of the Node.


{
  "plugin": "__main__",
  "node": "MyNode",
  "attrs": {
    "dimension": "time",
    "scale": 1.0
  }
}

## Setting Defaults
Notes, this method for handling defaults can be used instead of custom contructors or initializers for nodes

In [13]:
# Demonstrate when defaults are called
m1 = MyNode()
m2 = MyNode()
m3 = MyNode(scale=7)
m4 = MyNode(_other_attr=-4, scale=7)

# This line causes the default for scale to be computed based on the value of _other_attr
print ("m1 defaults:", m1._other_attr, m1.scale)  
# Changing _other_attr will not change m1.scale
m1._other_attr = -2  
print ("m1 _other_attr changed:", m1._other_attr, m1.scale)

# If we change _other_attr before asking for scale, the default for scale will change
m2._other_attr = -2
print ('m2 _other_attr changed, default scale:', m2._other_attr, m2.scale)

# If we set scale at instantiation of the class, the value of _other_attr will not affect scale
m3._other_attr = -3
print ('m3 _other_attr changed, scale set:', m3._other_attr, m3.scale)

# Same goes for instantiation of both when the class is created
print ('m4 set:', m4._other_attr, m4.scale)

m1 defaults: -1.0 1.0
m1 _other_attr changed: -2.0 1.0
m2 _other_attr changed, default scale: -2.0 4.0
m3 _other_attr changed, scale set: -3.0 7.0
m4 set: -4.0 7.0


## Demonstration of Traitlets Observations
This shows how a function is called whenever an 'observed' attribute is modified.

In [14]:
my_node = MyNode(dimension='lat')
my_node

Changing extracted dimension from time to lat


TraitError: The 'input' trait of a MyNode instance must be a CoordData, but a value of class 'NoneType' (i.e. None) was specified.

# Trailets Basics
Traitlets is a pure Python library used to add `traits` or `typing` back into Python. Python is inherently a `type-less` language, and relies or so-called `duck-typing` to determine the type of a variable. For example, Python does not discriminate between a string and an integer or a float. Traitlets allows Python programmers to ensure that their class attributes have the correct type. 

Among `traitlets` features, PODPAC uses the:
* Typing of class attributes
    * This includes simple Typing (Int, Float, Unicode, Dict, etc.)
    * But also more complex typing (correct instance of a class, with the correct dimensions, etc.).
* Ability to set defaults for attributes
* Tagging of class attributes
* Observing a class attribute for changes 

## Creating a traited class

In [9]:
# Define the class
class MyClass(tl.HasTraits):
    floating_point = tl.Float()
    integer = tl.Int()
    string = tl.Unicode()

    # Shortcut function used to print values of the class
    def __repr__(self):
        print_str = []
        for name in self.trait_names():
            print_str.append(name + ': ' + str(getattr(self, name)))
        return '\n'.join(print_str)
    
# Create an instance of the class
my_class = MyClass()

In [10]:
# Traits have defaults
my_class

floating_point: 0.0
integer: 0
string: 

In [11]:
# We can assign values to the traits that match their type
my_class.integer = 1
my_class.floating_point = 2.1
my_class.string = 'abc'
my_class

floating_point: 2.1
integer: 1
string: abc

In [23]:
# but if the type doesn't match, an exception is thrown
try: 
    my_class.floating_point = '1'
except tl.TraitError as e:
    print(e)
try: 
    my_class.integer = 1.1
except tl.TraitError as e:
    print(e)
try: 
    my_class.string = 8.67
except tl.TraitError as e:
    print(e)

The 'floating_point' trait of a MyClass instance must be a float, but a value of '1' <class 'str'> was specified.
The 'integer' trait of a MyClass instance must be an int, but a value of 1.1 <class 'float'> was specified.
The 'string' trait of a MyClass instance must be a unicode string, but a value of 8.67 <class 'float'> was specified.


**Note**: The integer trait threw an exception when we provided a floating point number. 

If you just want to round the integer, you can use the 'Cast' version of a trait.

In [21]:
# Define a new class that inherits from the previous class
class MyClass2(MyClass):
    cast_integer = tl.CInt()  # Create a trait with a cast integer version
    string = tl.CUnicode()    # Over-write the unicode trait with the cast version
my_class2 = MyClass2()
my_class2

cast_integer: 0
floating_point: 0.0
integer: 0
string: 

In [22]:
my_class2.cast_integer = 1.1
my_class2.string = 8.76
my_class2

cast_integer: 1
floating_point: 0.0
integer: 0
string: 8.76

### Class Instance Traits

In [31]:
class MyClass3(MyClass):
    my_class2 = tl.Instance(MyClass2)
    my_class = tl.Instance(MyClass)
my_class3 = MyClass3()
my_class3.my_class2 = MyClass2()
my_class3.my_class = MyClass2()
my_class3

floating_point: 0.0
integer: 0
my_class: cast_integer: 0
floating_point: 0.0
integer: 0
string: 
my_class2: cast_integer: 0
floating_point: 0.0
integer: 0
string: 
string: 

In [33]:
my_class3.my_class = MyClass2()  # Can assign child classes (still an instance of MyClass)
try: 
    my_class3.my_class2 = MyClass()  # Cannot assign child classes (MyClass is not an instance of MyClass2)
except tl.TraitError as e:
    print(e)

The 'my_class2' trait of a MyClass3 instance must be a MyClass2, but a value of class '__main__.MyClass' (i.e. floating_point: 0.0
integer: 0
string: ) was specified.


## Initializing Traits

In [ ]:
# TODO

## Tagging Attributes


In [36]:
# TODO

## Observing Traits

# PODPAC and Traits

In [ ]:
# TODO